In [8]:
# 06 Mar  2024
# Protection Tag removed
# Zip all the file structure
# Change Ext to excel like .xlsx or .xlsm
# Keep the source file backup
# Keep the source file extension
# Added Process Log file for check what happening in the process



#------------------ Install and import necessary tools.--------------------------

#!pip install openpyxl
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import openpyxl
import shutil
import zipfile
import os


# ------------------ Redirecting print statements to a log file ------------------
log_file_path = "process_log.txt"

def log_message(message):
    with open(log_file_path, 'a') as log_file:
        log_file.write(message + '\n')


# # ------------------ Select the Source Excel File -------------------------------

# Create a Tkinter root window (it won't be shown)
root = Tk()
root.withdraw()

# Ask the user to select an Excel file using Windows Explorer
excel_file_path = askopenfilename(filetypes=[("Excel Files", "*.xlsx;*.xlsm")])

# Check if the user selected a file
if excel_file_path:
    # Load the Excel workbook using openpyxl
    workbook = openpyxl.load_workbook(excel_file_path)

    # Get the number of worksheets in the workbook
    num_sheets = len(workbook.sheetnames)
    print(f"Number of sheets in the workbook: {num_sheets}")
    log_message(f"Number of sheets in the workbook: {num_sheets}")
    # Get the sheet names in the workbook
    sheet_names = workbook.sheetnames
    print(f"Sheet names in the workbook: {sheet_names}")
    log_message(f"Sheet names in the workbook: {sheet_names}")
    

    # Check if each sheet is protected or not
    for sheet_name in sheet_names:
        sheet = workbook[sheet_name]
        if sheet.protection.sheet:
            print(f"{sheet_name} is protected.")
            log_message(f"{sheet_name} is protected.")
        else:
            print(f"{sheet_name} is not protected.")
            log_message(f"{sheet_name} is not protected.")
            
# # --------------- Keep Backup of the Source Excel File -------------------


    # Create a backup file name with the same extension as the original file
    base_name, extension = os.path.splitext(excel_file_path)
    backup_file_path = f"{base_name}_backup{extension}"

    # Copy the original file to the backup file
    shutil.copy(excel_file_path, backup_file_path)

    print(f"Backup file created: {backup_file_path}")
    log_message(f"Backup file created: {backup_file_path}")
    
    
# # --------------- Convert the Source Excel File to Zip file -------------------   
    
# Check if the user selected a file
if backup_file_path:
    # Replace the extension with '.zip'
    zip_file_path = os.path.splitext(backup_file_path)[0] + '.zip'

    # Rename the file by moving it to a new path
    os.rename(backup_file_path, zip_file_path)

    print(f"Excel file '{backup_file_path}' has been renamed to '{zip_file_path}'.")
    log_message(f"Excel file '{backup_file_path}' has been renamed to '{zip_file_path}'.")
else:
    print("No Excel file selected.")
    log_message("No Excel file selected.")
            

        
    
# # --------------- Extract the Zip file -----------------------------------------  


# Check if the user selected a file
if zip_file_path:
    # Create a folder with the same name as the zip file (excluding the extension)
    folder_name = os.path.splitext(os.path.basename(zip_file_path))[0]
    os.makedirs(folder_name, exist_ok=True)

    # Create a ZipFile object
    with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
        # Extract the contents to the folder
        zip_file.extractall(folder_name)

    print(f"Contents of '{zip_file_path}' have been extracted to the folder '{folder_name}'.")
    log_message(f"Contents of '{zip_file_path}' have been extracted to the folder '{folder_name}'.")
else:
    print("No zip file selected.")
    log_message("No zip file selected.")


#-------------------- check how many xml file created------------------------------    
    
import os

# Specify the path to the extracted folder
folder_path = folder_name

# Construct the full path to the "xl/worksheets/" folder
worksheets_folder = os.path.join(folder_path, "xl/worksheets/")

# Get all XML files in the "xl/worksheets/" folder
xml_files = [f for f in os.listdir(worksheets_folder) if f.endswith('.xml')]

# Print the names of all XML files
print("XML files in xl/worksheets/ folder:")
log_message("XML files in xl/worksheets/ folder:")
for xml_file in xml_files:
    print(xml_file)
    log_message(xml_file)

#-------------------- check which xml file contain protection tag or not --------------    



 # --------------------- replace the tag-------------------------------


import os
import re

# Specify the path to the extracted folder
folder_path = folder_name

# Construct the full path to the "xl/worksheets/" folder
worksheets_folder = os.path.join(folder_path, "xl/worksheets/")

# Get all XML files in the "xl/worksheets/" folder
xml_files = [f for f in os.listdir(worksheets_folder) if f.endswith('.xml')]

# Text to search for in each XML file
tag_to_search = re.compile(r'<sheetProtection[^>]*>')

# Text to replace the found tag
new_text = ''

# Function to replace text in a file
def replace_text_in_file(file_path, old_text_pattern, new_text):
    # Read the content of the original file
    with open(file_path, 'r') as file:
        content = file.read()

    # Replace the specified text using a regular expression pattern
    content = old_text_pattern.sub(new_text, content)

    # Write the modified content back to the original file
    with open(file_path, 'w') as file:
        file.write(content)

# Check each XML file for the specified tag and replace if found
for xml_file in xml_files:
    xml_file_path = os.path.join(worksheets_folder, xml_file)

    with open(xml_file_path, 'r') as file:
        content = file.read()

        if tag_to_search.search(content):
            print(f"Replacing '<sheetProtection>' tag in {xml_file}...")
            log_message(f"Replacing '<sheetProtection>' tag in {xml_file}...")
            replace_text_in_file(xml_file_path, tag_to_search, new_text)
            print(f"Replacement complete for {xml_file}.")
            log_message(f"Replacement complete for {xml_file}.")
        else:
            print(f"{xml_file} does not contain the '<sheetProtection>' tag. No replacement needed.")
            log_message(f"{xml_file} does not contain the '<sheetProtection>' tag. No replacement needed.")

            
            
            
 # ---------------------- zipping the unprotected files------------------           
            
import shutil
import os

# Specify the path to the extracted folder
folder_path = folder_name

# Zip the folder
zip_folder_path = shutil.make_archive(folder_path, 'zip', folder_path)

# Rename the zip file
new_zip_folder_path = f"Unprotected_{folder_path}.zip"
os.rename(zip_folder_path, new_zip_folder_path)

print(f"Folder '{folder_path}' has been zipped and renamed to '{new_zip_folder_path}'.")
log_message(f"Folder '{folder_path}' has been zipped and renamed to '{new_zip_folder_path}'.")



#----------------------------- change the extenstion to excel file ------------


# Check if the user selected a file
if new_zip_folder_path:
    # Replace the extension with '.zip'
    unprotected_excel_file_path = f"{base_name}_Unprotected{extension}"
    
    # unprotected_excel_file_path = os.path.splitext(new_zip_folder_path)[0] + '.xlsx'

    # Rename the file by moving it to a new path
    os.rename(new_zip_folder_path, unprotected_excel_file_path)

    print(f"Unprotected Zipped file '{new_zip_folder_path}' has been renamed to '{unprotected_excel_file_path}'.")
    log_message(f"Unprotected Zipped file '{new_zip_folder_path}' has been renamed to '{unprotected_excel_file_path}'.")
else:
    print("No Excel file selected.")
    log_message("No Excel file selected.")



# Example usage:
# log_message(f"Number of sheets in the workbook: {num_sheets}")
# log_message(f"Sheet names in the workbook: {sheet_names}")
# log_message(f"{sheet_name} is protected.")
# log_message(f"{sheet_name} is not protected.")
# log_message(f"Backup file created: {backup_file_path}")
# log_message(f"Excel file '{backup_file_path}' has been renamed to '{zip_file_path}'.")
# log_message(f"Contents of '{zip_file_path}' have been extracted to the folder '{folder_name}'.")
# ... (add log_message for other print statements)

# Close the log file
log_message("Process completed.")
    

Number of sheets in the workbook: 40
Sheet names in the workbook: ['Sheet1', 'Sheet10', 'msg (7)', 'Production Details by Shift (2)', 'msg (6)', 'msg (5)', 'msg (4)', 'msg (3)', 'msg', 'Production Details by Shift', 'KPI DutyReg', 'KPI Burs&Power', 'KPI Prod', 'KPI Weight', 'KPI Safety', 'KPI ShortExc', 'User', 'Admin', 'SCBL Production Report', 'SCBL P Report Print', 'SCBL Production Details', 'SCBL Batch', 'SCBL Trawler', 'SCBL Trawler Rough', 'Production Report', 'P Report Print', 'Production Details', 'Monthly Summery', 'Monthly', 'Bursting', 'Sheet4', 'Gold Preparation', 'Bag Stock', 'Bulk', 'Sheet3', 'Sheet2', 'Prod & Burst', 'Bursting & Civil', 'Sheet5', 'Sheet6']
Sheet1 is not protected.
Sheet10 is not protected.
msg (7) is not protected.
Production Details by Shift (2) is not protected.
msg (6) is not protected.
msg (5) is not protected.
msg (4) is not protected.
msg (3) is not protected.
msg is not protected.
Production Details by Shift is not protected.
KPI DutyReg is not pr

UnicodeDecodeError: 'charmap' codec can't decode byte 0x90 in position 789334: character maps to <undefined>